# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
# Setting this to 'False' switches to alternate experimental code
# and requires the file './data/simpsons/simpsons_script_lines_sorted.csv'
pass_unittests = True 

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
if not pass_unittests:
    import helper_new as helper
else:
    import helper

    data_dir = './data/simpsons/moes_tavern_lines.txt'
    text = helper.load_data(data_dir)
    # Ignore notice, since we don't use it for analysing the data
    text = text[81:]

In [2]:
if not pass_unittests:
    # Create curated data
    file = 'simpsons.p'
    helper.create_simpsons(file)

    # Load curated data
    (word_count, episode_id,
    character_id, raw_character_text,
    location_id, raw_location_text,
    spoken_words,
    int_to_character, character_to_int) = helper.load_params_jj(file)

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
import numpy as np

if not pass_unittests:
    n = 10 # number of lines
    rint = np.random.randint(0,len(spoken_words)-n)
    view_sentence_range = (rint, rint+n)
    
    print('Dataset Stats')
    print('Number of episodes: {}'.format(len(set(episode_id))))
    print('Roughly the number of unique words: {}'.format(len({word: None for n in range(len(spoken_words)) 
                                                               for word in spoken_words[n].split()})))
    scenes_ind = []
    num = 0
    for n, i in enumerate(location_id):
        if i != num: # New scene
            scenes_ind.append(n)
        num = i
    print('Number of scenes: {}'.format(len(scenes_ind)))
    average_lines_count_scene = len(spoken_words) / len(scenes_ind)
    print('Average number of lines in each scene: {}'.format(average_lines_count_scene))

    print('Number of lines: {}'.format(len(spoken_words)))
    word_count_line = [len(line.split()) for line in spoken_words]
    print('Average number of words in each line: {}'.format(np.average(word_count_line)))

    print()
    print('The sentences {} to {}:'.format(*view_sentence_range))
    print('\n'.join([ '{}: {}'.format(raw_character_text[n],spoken_words[n]) for
                                    n in range(view_sentence_range[0],view_sentence_range[1])]))
else:
    view_sentence_range = (0, 10)

    """
    DON'T MODIFY ANYTHING IN THIS CELL
    """
    # import numpy as np

    print('Dataset Stats')
    print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
    scenes = text.split('\n\n')
    print('Number of scenes: {}'.format(len(scenes)))
    sentence_count_scene = [scene.count('\n') for scene in scenes]
    print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

    sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
    print('Number of lines: {}'.format(len(sentences)))
    word_count_sentence = [len(sentence.split()) for sentence in sentences]
    print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

    print()
    print('The sentences {} to {}:'.format(*view_sentence_range))
    print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True) # Words sorted from most common to least
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}     # Careful, Later the assumption is made that 
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}        # max(int_to_vocab.values()) == len(int_to_vocab.values())
    
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokens = ['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n']
    label = 'tokentext'
    token_dict = {tokens[n]: label.join(str(n)) for n in range(len(tokens))}
    
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
if pass_unittests:
    tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
if not pass_unittests:
    num_scenes = 2500 # Number of scenes to train on (one line scenes are removed)
    location_counts = Counter(location_id)
    print(location_counts.most_common(10))
    location = 5 # Moes Tavern (15) Simpson Home (5)
    ind = np.argwhere(location_id == location)
    count_scenes = 0
    scene_start = np.array([ind[0]], dtype=int)
    scene_end = np.array([], dtype=int)
    for n in range(len(ind)-1):
        if ind[n+1] - ind[n] != 1:
            count_scenes += 1
            scene_end = np.append(scene_end, ind[n])
            scene_start = np.append(scene_start, ind[n+1])
    scene_end = np.append(scene_end, ind[-1])
    print(count_scenes)    
    assert num_scenes <= count_scenes,\
        'Number of scenes exceeds maximum: {}'.format(count_scenes)

    ind_s = np.sort(np.random.choice(np.arange(len(scene_start)), num_scenes, replace=False))
    # Remove one line scenes
    ind_s = ind_s[np.where((scene_end[ind_s] - scene_start[ind_s]) > 1)]
    ind = []
    print(len(ind_s))
    _ = [ ind.extend(list(range(scene_start[ind_s[n]],scene_end[ind_s[n]]))) for n in range(len(ind_s))]
    text = np.concatenate((raw_character_text[ind, None],spoken_words[ind, None]), axis=1)
    
    scene_marker = (np.cumsum(scene_end[ind_s] - scene_start[ind_s]) - 1).tolist()
 
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
# helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)
if pass_unittests:
    helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)
else:
    helper.preprocess_and_save_data(text, scene_marker, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
pass_unittests = True
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
if pass_unittests:
    import helper
else:
    import helper_new as helper
import problem_unittests as tests

if pass_unittests:
    int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
else:
    int_text, scene_marker, vocab_to_int, int_to_vocab = helper.load_preprocess()

In [8]:
# For testing
if not pass_unittests:
    for n in range(20):
        print(helper.remove_token_spaces(' '.join([int_to_vocab[ii] for ii in int_text[n]])))
        if n in scene_marker:
            print('\n\n')

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [10]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs_ = tf.placeholder(tf.int32, [None, None], name='input')
    targets_ = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    if pass_unittests:
        return (inputs_, targets_, learning_rate)
    else:
        return (inputs_, targets_, learning_rate, keep_prob)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
if pass_unittests:
    tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [11]:
def get_init_cell(batch_size, rnn_size, keep_prob=1.0):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm_layers = 2
    
    # Basic LSTM cell with dropout stacked 'lstm_layers' times
    cell = tf.contrib.rnn.MultiRNNCell(
        [tf.contrib.rnn.DropoutWrapper(
            tf.contrib.rnn.BasicLSTMCell(rnn_size), output_keep_prob=keep_prob)
         for _ in range(lstm_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)    
    initial_state = tf.identity(initial_state, name='initial_state')
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
if pass_unittests:
    tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [13]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    Outputs, FinalState = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(FinalState, name='final_state')
    
    return (Outputs, FinalState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [14]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    Outputs, FinalState = build_rnn(cell, embed)
    Logits = tf.contrib.layers.fully_connected(Outputs, vocab_size, activation_fn=None) # [:, -1]
    return (Logits, FinalState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [15]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    if pass_unittests:
        n_batches = (len(int_text) - 1)//(batch_size*seq_length)
        # shape = (number of batches, 2, batch size, sequence length)
        batches = np.ndarray(shape=(batch_size, 2, n_batches, seq_length), dtype=int)
        shape = (batch_size, n_batches, seq_length)
        batches[:,0,:,:] = np.array(
            int_text[0:n_batches*batch_size*seq_length]).reshape(shape)
        batches[:,1,:,:] = np.array(
            int_text[1:n_batches*batch_size*seq_length+1]).reshape(shape)
        batches = np.transpose(batches, axes=(2,1,0,3))
        batches[-1,-1,-1,-1] = batches[0,0,0,0]
    else:
        # Sequence length is trimed words per line length. 
        # Example:
        # line: "Jason: Hello my Name is Jason . "
        # line with seq_length = 4: "Jason: Hello my Name"
        # line with seq_length = 9: 
        #     "Jason: 'placeholder' 'placeholder' Hello my Name is Jason ."
        '''
        features = np.zeros((len(int_text), seq_length), dtype=int)
        for i, row in enumerate(int_text):
            features[i, -len(row):] = np.array(row)[-seq_length:]
            features[i, 0] = row[0]
        n_batches = (len(features) - 1)//(batch_size)
        # shape = (number of batches, 2, batch size, sequence length)
        batches = np.ndarray(shape=(n_batches, 2, batch_size, seq_length), dtype=int)
        shape = (n_batches, batch_size, seq_length)
        batches[:,0,:,:] = np.array(
            features[0:n_batches*batch_size,:]).reshape(shape)
        batches[:,1,:,:] = np.array(
            features[1:n_batches*batch_size+1,:]).reshape(shape)
        '''
        int_text_flat = [item for sublist in int_text for item in sublist]
        n_batches = (len(int_text_flat) - 1)//(batch_size*seq_length)
        # shape = (number of batches, 2, batch size, sequence length)
        batches = np.ndarray(shape=(n_batches, 2, batch_size, seq_length), dtype=int)
        shape = (n_batches, batch_size, seq_length)
        batches[:,0,:,:] = np.array(
            int_text_flat[0:n_batches*batch_size*seq_length]).reshape(shape)
        batches[:,1,:,:] = np.array(
            int_text_flat[1:n_batches*batch_size*seq_length+1]).reshape(shape)
        
    return batches

#test = get_batches(list(range(1,21)), 3, 2)
#test = get_batches(int_text, 5, 5)
#print(test)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
if pass_unittests:
    tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [19]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 128 # 256
# RNN Size
rnn_size = 512 # 256 
# Embedding Dimension Size
embed_dim = 128
# Sequence Length
seq_length = 25
# Learning Rate
learning_rate = 0.01
# Keep Probability
keep_probability = 1.0
# Show stats for every n number of batches
show_every_n_batches = 50


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    if pass_unittests:
        input_text, targets, lr = get_inputs()
    else:
        input_text, targets, lr, keep_prob = get_inputs()
    input_data_shape = tf.shape(input_text)
    if pass_unittests:
        cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    else:
        cell, initial_state = get_init_cell(input_data_shape[0], rnn_size, keep_prob)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})
        ind = np.arange(len(batches))
        np.random.shuffle(ind)
        #for batch_i, (x, y) in enumerate(batches):
        for batch_i in range(len(batches)):
            (x, y) = batches[ind[batch_i]]
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            if not pass_unittests:
                feed[keep_prob] = keep_probability
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.822
Epoch   2 Batch    8/21   train_loss = 6.332
Epoch   4 Batch   16/21   train_loss = 6.093
Epoch   7 Batch    3/21   train_loss = 5.438
Epoch   9 Batch   11/21   train_loss = 5.091
Epoch  11 Batch   19/21   train_loss = 4.458
Epoch  14 Batch    6/21   train_loss = 3.891
Epoch  16 Batch   14/21   train_loss = 3.545
Epoch  19 Batch    1/21   train_loss = 2.875
Epoch  21 Batch    9/21   train_loss = 2.516
Epoch  23 Batch   17/21   train_loss = 2.181
Epoch  26 Batch    4/21   train_loss = 1.726
Epoch  28 Batch   12/21   train_loss = 1.581
Epoch  30 Batch   20/21   train_loss = 1.289
Epoch  33 Batch    7/21   train_loss = 0.953
Epoch  35 Batch   15/21   train_loss = 0.735
Epoch  38 Batch    2/21   train_loss = 0.564
Epoch  40 Batch   10/21   train_loss = 0.475
Epoch  42 Batch   18/21   train_loss = 0.387
Epoch  45 Batch    5/21   train_loss = 0.240
Epoch  47 Batch   13/21   train_loss = 0.240
Epoch  50 Batch    0/21   train_loss = 0.154
Epoch  52 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [23]:
pass_unittests = True
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
if pass_unittests:
    import helper
else:
    import helper_new as helper
import problem_unittests as tests

# _, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
if pass_unittests:
    _, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
    seq_length, load_dir = helper.load_params()
else:
    _, scene_marker, vocab_to_int, int_to_vocab = helper.load_preprocess()
    seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [24]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor        = loaded_graph.get_tensor_by_name(name="input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name(name="initial_state:0")
    FinalStateTensor   = loaded_graph.get_tensor_by_name(name="final_state:0")
    ProbsTensor        = loaded_graph.get_tensor_by_name(name="probs:0")
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [25]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    ind = np.argmax(probabilities) # index of maximum value
    word = int_to_vocab[ind]
    
    return word 


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [26]:
#print(int_to_vocab.values())
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
if pass_unittests:
    prime_word = 'moe_szyslak'
else:
    prime_word = '\nMoe Szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    if not pass_unittests:
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        if pass_unittests:
            feed={input_text: dyn_input, initial_state: prev_state}
        else:
            feed={input_text: dyn_input, initial_state: prev_state, loaded_keep_prob: 1.0}
        probabilities, prev_state = sess.run(
            [probs, final_state],
            feed_dict=feed)
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)
        #if not pass_unittests:
        #    pass
        gen_sentences.append(pred_word)

    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    if pass_unittests:
        for key, token in token_dict.items():
            ending = ' ' if key in ['\n', '(', '"'] else ''
            tv_script = tv_script.replace(' ' + token.lower(), key)
        tv_script = tv_script.replace('\n ', '\n')
        tv_script = tv_script.replace('( ', '(')
    else:
        tv_script = helper.remove_token_spaces(tv_script)
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(uneasy) oh, i can't wait till you guys get to uh...
thought_bubble_lenny: yep, that's what we'd do? we'd like to get edna out of you, you can say you could go home and take any dog instead of a hundred and i'd her man that's too.
marge_simpson: aw, that's a lot about human nature by watching things these people i know.


moe_szyslak: ah, okay, what happened?
lenny_leonard:(into phone) it's you, you want no.
moe_szyslak: i don't know why i just gonna win.
moe_szyslak: oh, no. this is the moment.
homer_simpson: yep. you know this is a little bit, duff" who gave" the real two" accurate 'cause" kent to do with the...(then) yeah, but, uh, can you think it would have.
carl_carlson: he got that character is a beer?
barney_gumble: oh, wait, look at me! i'm a problem? that's the best friend you've got to


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.